<a href="https://colab.research.google.com/github/MihneaGherghel/ColabCopy/blob/main/FakeNews2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import re
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
class OptainData:
  def __init__(self,file_path):
    self.data = pd.read_csv(file_path)
    self.data['label'] = pd.to_numeric(self.data['label'], errors='coerce')
    self.data = self.data.dropna()
    self.data['label'] = self.data['label'].astype(int)
    self.data.reset_index(inplace=True)

  def split_labels_samples(self):
    samples=self.data.drop('label',axis=1)
    labels=self.data['label']
    return samples,labels,self.data

In [3]:
data=OptainData('./gdrive/MyDrive/train.csv/FakeNews.csv')
samples,labels,samples_labels=data.split_labels_samples()

In [7]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [8]:
messages=samples_labels.copy()
messages.reset_index(inplace=True)
corpus=[]
ps=PorterStemmer()
for i in range(0,len(messages)):
  review=re.sub('[^a-zA-Z]',' ', messages['title'][i])
  review=review.lower()
  review=review.split()
  review=[ps.stem(word) for word in review if not word in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

In [33]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout

In [20]:
voc_size=5000
onehot_repr=[one_hot(words,voc_size) for words in corpus]

In [21]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)


In [79]:
embedding_vector_features=5
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(20))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [80]:
import numpy as np
from sklearn.model_selection import train_test_split

In [81]:
X_final=np.array(embedded_docs)
y_final=np.array(messages['label'])


In [82]:
X_train, X_test, y_train, y_test= train_test_split(X_final, y_final, test_size=0.33,random_state=42)

In [83]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10, batch_size=64)

Epoch 1/10
192/192 [==============================] - 8s 26ms/step - loss: 0.5253 - accuracy: 0.7387 - val_loss: 0.2934 - val_accuracy: 0.8853
Epoch 2/10
192/192 [==============================] - 3s 17ms/step - loss: 0.2228 - accuracy: 0.9172 - val_loss: 0.2036 - val_accuracy: 0.9148
Epoch 3/10
192/192 [==============================] - 3s 14ms/step - loss: 0.1529 - accuracy: 0.9469 - val_loss: 0.1980 - val_accuracy: 0.9210
Epoch 4/10
192/192 [==============================] - 3s 13ms/step - loss: 0.1209 - accuracy: 0.9585 - val_loss: 0.1951 - val_accuracy: 0.9190
Epoch 5/10
192/192 [==============================] - 3s 15ms/step - loss: 0.0968 - accuracy: 0.9686 - val_loss: 0.2264 - val_accuracy: 0.9178
Epoch 6/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0851 - accuracy: 0.9731 - val_loss: 0.2531 - val_accuracy: 0.9160
Epoch 7/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0689 - accuracy: 0.9786 - val_loss: 0.2614 - val_accuracy: 0.9142